# Ціль: Знайомство з об’єктами СХЕМИ БАЗИ ДАНИХ 
## Створення і використання тригерів


In [2]:
USE Accounting;

Commands completed successfully.

Total execution time: 00:00:00.309

In [3]:
-- тригер на заборону зміни оцінки окрім 'Fx'
CREATE TRIGGER DeleteResults
ON Results FOR DELETE
AS
IF NOT EXISTS (SELECT mark FROM deleted
WHERE mark='Fx' )
BEGIN
RAISERROR('Неможна виправляти оцінку!!!',16,1)  
-- Відмова транзакції у випадку помилки
ROLLBACK TRAN
END;

Commands completed successfully.

Total execution time: 00:00:00.132

In [4]:
DROP TRIGGER DeleteResults;

Commands completed successfully.

Total execution time: 00:00:00.071

In [5]:
--сумарна таблиця всіх відмінників в першому семестрі 
CREATE VIEW StA
AS
SELECT s.name,s.surname,g.group_id FROM Results r
INNER JOIN Students s 
ON s.student_id=r.student_id
INNER JOIN Groups g
ON s.group_id=g.group_id
WHERE term=1 and mark='5';

Commands completed successfully.

Total execution time: 00:00:00.055

In [6]:
SELECT * FROM StA;

(3 rows affected)

Total execution time: 00:00:00.433

name,surname,group_id
Ivan,ivanov,1
daryna,kulyba,2
vasyl,symonenko,1


In [7]:
-- тригер на view : перевірка чи існує такий студент,якщо так,то поставити оцінку '5' інакше вивести повідомлення
CREATE TRIGGER InsteadOfInsert
ON StA
INSTEAD OF INSERT
AS 
BEGIN
IF NOT EXISTS(SELECT i.name,i.surname,i.group_id
FROM inserted i
INNER JOIN Students s
ON s.group_id = i.group_id
WHERE s.name = i.name AND s.surname = i.surname)
BEGIN
RAISERROR('Немає такого студента',16,1)  
-- Відмова транзакції у випадку помилки
ROLLBACK TRAN
END
ELSE
BEGIN
UPDATE Results 
SET mark = '5'
WHERE student_id = (SELECT s.student_id
FROM Students s 
INNER JOIN inserted i
ON s.group_id = i.group_id
WHERE s.name = i.name AND s.surname = i.surname)
END
END;


Commands completed successfully.

Total execution time: 00:00:00.065

In [8]:
DROP TRIGGER InsteadOfInsert;

Commands completed successfully.

Total execution time: 00:00:00.017